In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split

def train_cross_modality_centered(
    train_modality: str,
    test_modality: str,
    duration_s: int,
    cfg: dict = None
):
    # 1) Hyperparameters
    defaults = {"n_estimators": 100, "max_depth": 10, "random_state": 42}
    params = defaults.copy()
    if cfg:
        params.update(cfg)

    # 2) Path to feature file
    def feature_path(mod):
        return f"./features/fixed_really/balanced/{mod}/features_{mod}_{duration_s}s_balanced.csv"

    # 3) Load data
    if train_modality == test_modality:
        df = pd.read_csv(feature_path(train_modality))
        df_tr, df_te = train_test_split(df, test_size=0.25, stratify=df["label"], random_state=params["random_state"])
    else:
        df_tr = pd.read_csv(feature_path(train_modality))
        df_te = pd.read_csv(feature_path(test_modality))

    # 4) Feature selection
    drop_cols = [
        "video_id","segment","participant","start_time","end_time","label",
        "original_cont_duration","duration_s","real_sample_count", "window_start_frame", "original_duration"
    ]
    feat_cols = sorted(set(df_tr.columns) - set(drop_cols))

    # 5) Construct matrices
    X_tr, y_tr = df_tr[feat_cols].fillna(0), df_tr["label"]
    X_te, y_te = df_te[feat_cols].fillna(0), df_te["label"]

    # 6) Train classifier
    clf = RandomForestClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        class_weight="balanced",
        random_state=params["random_state"],
        n_jobs=-1
    )
    clf.fit(X_tr, y_tr)
    preds = clf.predict(X_te)

    # 7) Metrics
    report = classification_report(y_te, preds, output_dict=True, zero_division=0)
    f1_macro = f1_score(y_te, preds, average="macro")
    report_1 = report.get("1", {"precision": 0, "recall": 0, "f1-score": 0})
    y_te_pos = (y_te == 1).sum()

    # 8) Feature importances
    importance_df = pd.DataFrame({
        "feature": feat_cols,
        "importance": clf.feature_importances_
    }).sort_values(by="importance", ascending=False)

    metrics = {
        "train_modality": train_modality,
        "test_modality":  test_modality,
        "duration_s":     duration_s,
        "accuracy":       report["accuracy"],
        "f1_macro":       f1_macro,
        "precision_1":    report_1["precision"],
        "recall_1":       report_1["recall"],
        "f1_1":           report_1["f1-score"],
        "y_te_positives": y_te_pos,
        "feature_importances": importance_df.reset_index(drop=True)
    }

    print(f"✅ {train_modality} → {test_modality} @ {duration_s}s | Positives in test set: {y_te_pos}")
    print("\n🔍 Top 10 most important features:")
    print(importance_df.head(10))
    return metrics


In [2]:
# %%
modalities = ["with_audio", "no_audio", "only_audio"]
durations  = [1, 2, 10]

results = []
for dur in durations:
    for train_mod in modalities:
        for test_mod in modalities:
            res = train_cross_modality_centered(
                train_modality=train_mod,
                test_modality=test_mod,
                duration_s=dur
            )
            results.append(res)

# Guardar resultados
df_results = pd.DataFrame(results)
df_results.to_csv("crossmod_centered_summary.csv", index=False)
df_results


✅ with_audio → with_audio @ 1s | Positives in test set: 7003

🔍 Top 10 most important features:
                      feature  importance
18                    corr_yz    0.074959
2   accelX_filtered_deriv_std    0.069893
12  accelZ_filtered_deriv_std    0.065920
17                    corr_xz    0.063784
13     accelZ_filtered_energy    0.058240
7   accelY_filtered_deriv_std    0.058056
0                         SMA    0.055742
8      accelY_filtered_energy    0.053562
15        accelZ_filtered_var    0.053317
16                    corr_xy    0.052490
✅ with_audio → no_audio @ 1s | Positives in test set: 27806

🔍 Top 10 most important features:
                      feature  importance
18                    corr_yz    0.074785
12  accelZ_filtered_deriv_std    0.070165
2   accelX_filtered_deriv_std    0.065262
17                    corr_xz    0.064851
7   accelY_filtered_deriv_std    0.062498
16                    corr_xy    0.057827
13     accelZ_filtered_energy    0.055560
15        a

,train_modality,test_modality,duration_s,accuracy,f1_macro,precision_1,recall_1,f1_1,y_te_positives,feature_importances
0,with_audio,with_audio,1,0.841687,0.702988,0.365410,0.791661,0.500023,7003,feature importance 0 ...
1,with_audio,no_audio,1,0.785920,0.563492,0.193602,0.360426,0.251898,27806,feature importance 0 ...
2,with_audio,only_audio,1,0.783421,0.555912,0.183629,0.338329,0.238054,27955,feature importance 0 ...
3,no_audio,with_audio,1,0.748626,0.545141,0.172563,0.398900,0.240910,28012,feature importance 0 ...
4,no_audio,no_audio,1,0.792623,0.655829,0.300816,0.810962,0.438848,6951,feature importance 0 ...
5,no_audio,only_audio,1,0.740496,0.521567,0.143234,0.320193,0.197928,27955,feature importance 0 ...
6,only_audio,with_audio,1,0.746512,0.538674,0.164563,0.376517,0.229026,28012,feature importance 0 ...
7,only_audio,no_audio,1,0.741103,0.523253,0.145348,0.325613,0.200981,27806,feature importance 0 ...
8,only_audio,only_audio,1,0.799837,0.669141,0.315537,0.856632,0.461195,6989,feature importance 0 ...
9,with_audio,with_audio,2,0.848841,0.717371,0.382640,0.834071,0.524609,7003,feature importance 0 ...


In [13]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score

def train_centered_test_balanced(
    modality: str,
    duration_s: int,
    cfg: dict = None,
):
    """
    Train using event-centered windows, test on balanced sliding windows
    (both from the same modality).
    """
    # Hiperparámetros
    defaults = {"n_estimators": 100, "max_depth": 10, "random_state": 42}
    params = defaults.copy()
    if cfg:
        params.update(cfg)

    # Paths
    path_train = f"./features/fixed/{modality}/features_{modality}_{duration_s}s.csv"
    path_test  = f"./features/fixed_really/{modality}/features_{modality}_{duration_s}s.csv"

    df_tr = pd.read_csv(path_train)
    df_te = pd.read_csv(path_test)

    # Drop metadata
    drop_cols = [
        "video_id", "segment", "participant", "start_time", "end_time",
        "label", "original_cont_duration", "duration_s", "real_sample_count",
        "expected_sample_count", "padding_applied"
    ]
    feat_cols = sorted(set(df_tr.columns) & set(df_te.columns) - set(drop_cols))

    # Features & labels
    X_tr, y_tr = df_tr[feat_cols].fillna(0), df_tr["label"]
    X_te, y_te = df_te[feat_cols].fillna(0), df_te["label"]

    # Entrenamiento
    clf = RandomForestClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        class_weight="balanced",
        random_state=params["random_state"],
        n_jobs=-1
    )
    clf.fit(X_tr, y_tr)
    preds = clf.predict(X_te)

    report = classification_report(y_te, preds, output_dict=True, zero_division=0)
    f1_macro = f1_score(y_te, preds, average="macro")
    report_1 = report.get("1", {"precision": 0, "recall": 0, "f1-score": 0})
    y_te_pos = (y_te == 1).sum()

    metrics = {
        "modality":    modality,
        "duration_s":  duration_s,
        "accuracy":    report["accuracy"],
        "f1_macro":    f1_macro,
        "precision_1": report_1["precision"],
        "recall_1":    report_1["recall"],
        "f1_1":        report_1["f1-score"],
        "y_te_positives": y_te_pos
    }
    print(f"✅ {modality} @ {duration_s}s → Positives in test set: {y_te_pos}")

    return metrics


In [14]:
# %%
modalities = ["with_audio", "only_audio", "no_audio"]
durations  = [1, 2, 10]

results = []
for mod in modalities:
    for dur in durations:
        res = train_centered_test_balanced(
            modality=mod,
            duration_s=dur,
        )
        results.append(res)

df_results = pd.DataFrame(results)
df_results.to_csv("centered_train_test_balanced_summary.csv", index=False)
df_results


✅ with_audio @ 1s → Positives in test set: 3608
✅ with_audio @ 2s → Positives in test set: 4767
✅ with_audio @ 10s → Positives in test set: 6413
✅ only_audio @ 1s → Positives in test set: 4546
✅ only_audio @ 2s → Positives in test set: 5992
✅ only_audio @ 10s → Positives in test set: 8200
✅ no_audio @ 1s → Positives in test set: 2444
✅ no_audio @ 2s → Positives in test set: 2005
✅ no_audio @ 10s → Positives in test set: 613


,modality,duration_s,accuracy,f1_macro,precision_1,recall_1,f1_1,y_te_positives
0,with_audio,1,0.982772,0.500376,0.018196,0.006375,0.009442,3608
1,with_audio,2,0.981601,0.497098,0.022222,0.001888,0.003480,4767
2,with_audio,10,0.977107,0.494210,0.000000,0.000000,0.000000,6413
3,only_audio,1,0.976512,0.500076,0.019048,0.008799,0.012037,4546
4,only_audio,2,0.976741,0.496565,0.029520,0.002670,0.004897,5992
5,only_audio,10,0.970517,0.492761,0.043478,0.000244,0.000485,8200
6,no_audio,1,0.983518,0.503578,0.016282,0.014730,0.015467,2444
7,no_audio,2,0.990200,0.500819,0.012195,0.004489,0.006562,2005
8,no_audio,10,0.997328,0.500673,0.007576,0.001631,0.002685,613


In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score

def train_segmentation_test_sliding(
    modality: str,
    duration_s: int,
    train_segmentation: str,  # "centered" or "padded"
    cfg: dict = None,
):
    """
    Train using either 'centered' or 'padded' segments, test on sliding windows
    (same modality and duration).
    """
    # Hyperparameters
    defaults = {"n_estimators": 100, "max_depth": 10, "random_state": 42}
    params = defaults.copy()
    if cfg:
        params.update(cfg)

    # Paths based on train segmentation
    if train_segmentation == "centered":
        path_train = f"./features/from_centered/{modality}/{duration_s}s/annotations_{modality}_centered_{duration_s}s.csv"
    elif train_segmentation == "padded":
        path_train = f"./features/fixed/{modality}/features_{modality}_{duration_s}s.csv"
    else:
        raise ValueError("train_segmentation must be 'centered' or 'padded'")

    path_test = f"./features/fixed_really/balanced/{modality}/features_{modality}_{duration_s}s_balanced.csv"

    # Load data
    df_train = pd.read_csv(path_train)
    df_test  = pd.read_csv(path_test)

    # Drop metadata columns
    drop_cols = [
        "video_id", "segment", "participant", "start_time", "end_time",
        "label", "original_cont_duration", "duration_s", "real_sample_count",
        "expected_sample_count", "padding_applied"
    ]
    feat_cols = sorted(set(df_train.columns) & set(df_test.columns) - set(drop_cols))

    # Features & labels
    X_tr, y_tr = df_train[feat_cols].fillna(0), df_train["label"]
    X_te, y_te = df_test[feat_cols].fillna(0), df_test["label"]

    # Train
    clf = RandomForestClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        class_weight="balanced",
        random_state=params["random_state"],
        n_jobs=-1
    )
    clf.fit(X_tr, y_tr)
    preds = clf.predict(X_te)

    # Evaluation
    report = classification_report(y_te, preds, output_dict=True, zero_division=0)
    f1_macro = f1_score(y_te, preds, average="macro")
    report_1 = report.get("1", {"precision": 0, "recall": 0, "f1-score": 0})
    y_te_pos = (y_te == 1).sum()

    metrics = {
        "modality":          modality,
        "duration_s":        duration_s,
        "train_segmentation": train_segmentation,
        "test_segmentation":  "sliding",
        "accuracy":          report["accuracy"],
        "f1_macro":          f1_macro,
        "precision_1":       report_1["precision"],
        "recall_1":          report_1["recall"],
        "f1_1":              report_1["f1-score"],
        "y_te_positives":    y_te_pos
    }

    print(f"✅ {train_segmentation} → sliding | {modality} @ {duration_s}s → Positives in test set: {y_te_pos}")

    return metrics


In [4]:
modalities = ["with_audio", "only_audio", "no_audio"]
durations  = [1, 2, 10]
train_strategies = ["centered", "padded"]

results = []
for train_seg in train_strategies:
    for mod in modalities:
        for dur in durations:
            res = train_segmentation_test_sliding(
                modality=mod,
                duration_s=dur,
                train_segmentation=train_seg,
            )
            results.append(res)

df_results = pd.DataFrame(results)
df_results.to_csv("segmentation_train_test_sliding_summary.csv", index=False)
df_results


✅ centered → sliding | with_audio @ 1s → Positives in test set: 28012
✅ centered → sliding | with_audio @ 2s → Positives in test set: 28012
✅ centered → sliding | with_audio @ 10s → Positives in test set: 28012
✅ centered → sliding | only_audio @ 1s → Positives in test set: 27955
✅ centered → sliding | only_audio @ 2s → Positives in test set: 139776
✅ centered → sliding | only_audio @ 10s → Positives in test set: 139776
✅ centered → sliding | no_audio @ 1s → Positives in test set: 27806
✅ centered → sliding | no_audio @ 2s → Positives in test set: 27806
✅ centered → sliding | no_audio @ 10s → Positives in test set: 27806
✅ padded → sliding | with_audio @ 1s → Positives in test set: 28012
✅ padded → sliding | with_audio @ 2s → Positives in test set: 28012
✅ padded → sliding | with_audio @ 10s → Positives in test set: 28012
✅ padded → sliding | only_audio @ 1s → Positives in test set: 27955
✅ padded → sliding | only_audio @ 2s → Positives in test set: 139776
✅ padded → sliding | only_aud

,modality,duration_s,train_segmentation,test_segmentation,accuracy,f1_macro,precision_1,recall_1,f1_1,y_te_positives
0,with_audio,1,centered,sliding,0.675623,0.495010,0.128459,0.387905,0.193003,28012
1,with_audio,2,centered,sliding,0.709347,0.498985,0.121756,0.306868,0.174340,28012
2,with_audio,10,centered,sliding,0.815566,0.541331,0.166962,0.211659,0.186673,28012
3,only_audio,1,centered,sliding,0.721279,0.509689,0.132384,0.321803,0.187594,27955
4,only_audio,2,centered,sliding,0.510492,0.469017,0.523789,0.231012,0.320619,139776
5,only_audio,10,centered,sliding,0.520261,0.475634,0.548641,0.228530,0.322660,139776
6,no_audio,1,centered,sliding,0.661243,0.486588,0.123110,0.389952,0.187139,27806
7,no_audio,2,centered,sliding,0.631380,0.473269,0.118566,0.417500,0.184683,27806
8,no_audio,10,centered,sliding,0.766881,0.537764,0.160341,0.314213,0.212331,27806
9,with_audio,1,padded,sliding,0.893302,0.485477,0.154836,0.015029,0.027399,28012
